In [22]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import preprocessing as scale
from sklearn.model_selection import train_test_split
from tqdm.notebook import tqdm
from sklearn import mixture
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.decomposition import PCA

from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC, LinearSVC
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, confusion_matrix, recall_score, f1_score, classification_report
import matplotlib.pyplot as plt
import seaborn as sns

In [23]:
input_file = ["a.csv" , "b.csv" , "c.csv" , "e.csv" ]
columns_to_drop_bi = ['proto', 'ip_src', 'ip_dst']


The Federated Learner:

Parameters:

x: The data to be decided (array)

clfs:  The binary classifiers;(List of Classifiers)

p_nodes: The proportion of the nodes;(List)

gms: Gassian Mixture Models (List of GMMs)



Binary=RF

Binary=FNN:

Binary = CNN

In [34]:
    # data = pd.read_csv('openSmile_resampled/overall.csv')

    # data = pd.read_csv('openSmile_raw/f.csv')

    data = pd.read_csv('openSmile_balanced/f.csv')

    # Split the data into features (X) and labels (y)
    X = data.drop('y', axis=1)
    y = data['y']

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    n_estimators_custom = 50
    max_depth_custom = 10
    min_samples_split_custom = 5
    criterion_custom = 'gini'

    clf = RandomForestClassifier(n_estimators=n_estimators_custom,
                                   max_depth=max_depth_custom,
                                   min_samples_split=min_samples_split_custom,
                                   criterion=criterion_custom,
                                   random_state=42)
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)

In [34]:
## Centralized_RF_learn

from sklearn.ensemble import RandomForestClassifier
def Centralized_RF_learn(input_file, columns_to_drop_bi, random_state,  test_size = 0.25):
    
    data_list = []
    for i in range(len(input_file)):    
    data = pd.read_csv(input_file[i])
        data.drop(columns = columns_to_drop_bi, inplace = True)
        data_list.append(data)

    rs = random_state

    n_samples = []
    x_train_list = []
    y_train_list = []
    x_test_list = []
    y_test_list = []
    y = [np.ones(len(data_list[i])) * (i + 1) for i in range(len(data_list))]
    for i in range(len(input_file)): # reading the data
        data0 = pd.read_csv(input_file[i])
        y[i][data0.is_abnormal == 0] = 0
        n_samples.append(len(y[i])) # the number of this node
        x = data_list[i]
        x = x.drop(columns='is_abnormal')
        x_train, x_test, y_train, y_test = train_test_split(x, y[i], test_size = test_size, random_state = rs)
        x_train_list.append(x_train)
        y_train_list.append(y_train)
        x_test_list.append(x_test)
        y_test_list.append(y_test)

    x_train_full = pd.concat(x_train_list, ignore_index=True)
    scaler = scale.StandardScaler().fit(x_train_full)

    for i in range(len(input_file)):
        x_train_list[i] = scaler.transform(x_train_list[i])

    total_n_samples = np.sum(n_samples)

    p_nodes =  np.array(n_samples) / total_n_samples


    for i in range(len(input_file)):
        x_test_list[i] = scaler.transform(x_test_list[i])

    x_test = np.vstack(x_test_list)
    y_test = np.hstack(y_test_list)
    x_train = np.vstack(x_train_list)
    y_train = np.hstack(y_train_list)

    classifier = KNeighborsClassifier()
    classifier.fit(x_train, y_train)

    prediction = classifier.predict(x_test)

    correct = prediction == y_test
    accuracy = np.mean(correct)

    accs = []
    for i in range(len(input_file)+1):
        if len(correct[y_test==i]) == 0:
            accs.append(0)
        else:
            accs.append(np.mean(correct[y_test==i]))

    recalls = []
    for i in range(len(input_file)+1):
        if len(correct[prediction==i]) == 0:
            recalls.append(0)
        else:
            recalls.append(np.mean(correct[prediction==i]))

    return recalls, accs 



def test_RF_centralized(epoch, input_file, columns_to_drop_bi):

   recall , precision = np.ones((epoch,len(input_file)+1))*0 , np.ones((epoch,len(input_file)+1))*0

   print('main process:')

   for rs in tqdm(range(epoch)):
      recalls, accs = Centralized_knn_learn(input_file = input_file, columns_to_drop_bi=columns_to_drop_bi, random_state=rs)
      recall[rs,:] = recalls
      precision[rs,:] = accs

   for i in range(len(input_file)+1):
      print('Precision mean=',np.mean(precision[:,i]))
      print('Precision std=',np.std(precision[:,i]))

   for i in range(len(input_file)+1):
      print('Recall mean of ',i,' =',np.mean(recall[:,i]))
      print('Recall std of ',i,' =',np.std(recall[:,i]))

In [35]:
test_RF_centralized(epoch=1, input_file=input_file, columns_to_drop_bi=columns_to_drop_bi)

main process:



Precision mean= 0.9986343062275636
Precision std= 0.0
Precision mean= 1.0
Precision std= 0.0
Precision mean= 1.0
Precision std= 0.0
Precision mean= 1.0
Precision std= 0.0
Precision mean= 0.9801434087148373
Precision std= 0.0
Recall mean of  0  = 0.9971946230274693
Recall std of  0  = 0.0
Recall mean of  1  = 1.0
Recall std of  1  = 0.0
Recall mean of  2  = 1.0
Recall std of  2  = 0.0
Recall mean of  3  = 0.999599278701663
Recall std of  3  = 0.0
Recall mean of  4  = 0.9908001115137999
Recall std of  4  = 0.0


In [36]:
## Centralized_FNN_learn

def Centralized_FNN_learn(input_file, columns_to_drop_bi, random_state,  test_size = 0.25):
    
    data_list = []
    for i in range(len(input_file)):
        data = pd.read_csv(input_file[i])
        data.drop(columns = columns_to_drop_bi, inplace = True)
        data_list.append(data)

    rs = random_state

    n_samples = []
    x_train_list = []
    y_train_list = []
    x_test_list = []
    y_test_list = []
    y = [np.ones(len(data_list[i])) * (i + 1) for i in range(len(data_list))]
    for i in range(len(input_file)): # reading the data
        data0 = pd.read_csv(input_file[i])
        y[i][data0.is_abnormal == 0] = 0
        n_samples.append(len(y[i])) # the number of this node
        x = data_list[i]
        x = x.drop(columns='is_abnormal')
        x_train, x_test, y_train, y_test = train_test_split(x, y[i], test_size = test_size, random_state = rs)
        x_train_list.append(x_train)
        y_train_list.append(y_train)
        x_test_list.append(x_test)
        y_test_list.append(y_test)

    x_train_full = pd.concat(x_train_list, ignore_index=True)
    scaler = scale.StandardScaler().fit(x_train_full)

    for i in range(len(input_file)):
        x_train_list[i] = scaler.transform(x_train_list[i])

    total_n_samples = np.sum(n_samples)

    p_nodes =  np.array(n_samples) / total_n_samples


    for i in range(len(input_file)):
        x_test_list[i] = scaler.transform(x_test_list[i])

    x_test = np.vstack(x_test_list)
    y_test = np.hstack(y_test_list)
    x_train = np.vstack(x_train_list)
    y_train = np.hstack(y_train_list)

    classifier = SVC(kernel = 'rbf', random_state = random_state + 1, gamma='scale',max_iter=-1, probability=True)
    classifier.fit(x_train, y_train)

    prediction = classifier.predict(x_test)

    correct = prediction == y_test
    accuracy = np.mean(correct)

    accs = []
    for i in range(len(input_file)+1):
        if len(correct[y_test==i]) == 0:
            accs.append(0)
        else:
            accs.append(np.mean(correct[y_test==i]))

    recalls = []
    for i in range(len(input_file)+1):
        if len(correct[prediction==i]) == 0:
            recalls.append(0)
        else:
            recalls.append(np.mean(correct[prediction==i]))

    return recalls, accs 


def test_FNN_centralized(epoch, input_file, columns_to_drop_bi):

   recall , precision = np.ones((epoch,len(input_file)+1))*0 , np.ones((epoch,len(input_file)+1))*0

   print('main process:')

   for rs in tqdm(range(epoch)):
      recalls, accs = Centralized_SVC_learn(input_file = input_file, columns_to_drop_bi=columns_to_drop_bi, random_state=rs)
      recall[rs,:] = recalls
      precision[rs,:] = accs

   for i in range(len(input_file)+1):
      print('Precision mean=',np.mean(precision[:,i]))
      print('Precision std=',np.std(precision[:,i]))

   for i in range(len(input_file)+1):
      print('Recall mean of ',i,' =',np.mean(recall[:,i]))
      print('Recall std of ',i,' =',np.std(recall[:,i]))


In [37]:
test_FNN_centralized(epoch=1, input_file=input_file, columns_to_drop_bi=columns_to_drop_bi)

main process:



Precision mean= 1.0
Precision std= 0.0
Precision mean= 1.0
Precision std= 0.0
Precision mean= 0.9907539118065434
Precision std= 0.0
Precision mean= 1.0
Precision std= 0.0
Precision mean= 0.9721456150027579
Precision std= 0.0
Recall mean of  0  = 0.9940653969977891
Recall std of  0  = 0.0
Recall mean of  1  = 1.0
Recall std of  1  = 0.0
Recall mean of  2  = 1.0
Recall std of  2  = 0.0
Recall mean of  3  = 1.0
Recall std of  3  = 0.0
Recall mean of  4  = 1.0
Recall std of  4  = 0.0


In [38]:
## Centralized_CNN_learn

def Centralized_CNN_learn(input_file, columns_to_drop_bi, random_state,  test_size = 0.25):
    
    data_list = []
    for i in range(len(input_file)):
        data = pd.read_csv(input_file[i])
        data.drop(columns = columns_to_drop_bi, inplace = True)
        data_list.append(data)

    rs = random_state

    n_samples = []
    x_train_list = []
    y_train_list = []
    x_test_list = []
    y_test_list = []
    y = [np.ones(len(data_list[i])) * (i + 1) for i in range(len(data_list))]
    for i in range(len(input_file)): # reading the data
        data0 = pd.read_csv(input_file[i])
        y[i][data0.is_abnormal == 0] = 0
        n_samples.append(len(y[i])) # the number of this node
        x = data_list[i]
        x = x.drop(columns='is_abnormal')
        x_train, x_test, y_train, y_test = train_test_split(x, y[i], test_size = test_size, random_state = rs)
        x_train_list.append(x_train)
        y_train_list.append(y_train)
        x_test_list.append(x_test)
        y_test_list.append(y_test)

    x_train_full = pd.concat(x_train_list, ignore_index=True)
    scaler = scale.StandardScaler().fit(x_train_full)

    for i in range(len(input_file)):
        x_train_list[i] = scaler.transform(x_train_list[i])

    total_n_samples = np.sum(n_samples)

    p_nodes =  np.array(n_samples) / total_n_samples


    for i in range(len(input_file)):
        x_test_list[i] = scaler.transform(x_test_list[i])

    x_test = np.vstack(x_test_list)
    y_test = np.hstack(y_test_list)
    x_train = np.vstack(x_train_list)
    y_train = np.hstack(y_train_list)

    classifier = LogisticRegression(max_iter=100)
    classifier.fit(x_train, y_train)

    prediction = classifier.predict(x_test)

    correct = prediction == y_test
    accuracy = np.mean(correct)

    accs = []
    for i in range(len(input_file)+1):
        if len(correct[y_test==i]) == 0:
            accs.append(0)
        else:
            accs.append(np.mean(correct[y_test==i]))

    recalls = []
    for i in range(len(input_file)+1):
        if len(correct[prediction==i]) == 0:
            recalls.append(0)
        else:
            recalls.append(np.mean(correct[prediction==i]))

    return recalls, accs 



def test_CNN_centralized(epoch, input_file, columns_to_drop_bi):

   recall , precision = np.ones((epoch,len(input_file)+1))*0 , np.ones((epoch,len(input_file)+1))*0

   print('main process:')

   for rs in tqdm(range(epoch)):
      recalls, accs = Centralized_LR_learn(input_file = input_file, columns_to_drop_bi=columns_to_drop_bi, random_state=rs)
      recall[rs,:] = recalls
      precision[rs,:] = accs

   for i in range(len(input_file)+1):
      print('Precision mean=',np.mean(precision[:,i]))
      print('Precision std=',np.std(precision[:,i]))

   for i in range(len(input_file)+1):
      print('Recall mean of ',i,' =',np.mean(recall[:,i]))
      print('Recall std of ',i,' =',np.std(recall[:,i]))


In [39]:
test_CNN_centralized(epoch=1, input_file=input_file, columns_to_drop_bi=columns_to_drop_bi)

main process:



Precision mean= 0.9985172467613548
Precision std= 0.0
Precision mean= 1.0
Precision std= 0.0
Precision mean= 0.9932432432432432
Precision std= 0.0
Precision mean= 1.0
Precision std= 0.0
Precision mean= 0.9721456150027579
Precision std= 0.0
Recall mean of  0  = 0.9946361940298507
Recall std of  0  = 0.0
Recall mean of  1  = 0.9997124784358827
Recall std of  1  = 0.0
Recall mean of  2  = 0.9937733499377335
Recall std of  2  = 0.0
Recall mean of  3  = 0.9993990384615384
Recall std of  3  = 0.0
Recall mean of  4  = 1.0
Recall std of  4  = 0.0


/Users/irohayachiyo/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
